# 01. LangFuse Setup and Tracing

## Mục tiêu học tập

Notebook này sẽ hướng dẫn bạn:
- Hiểu khái niệm và tầm quan trọng của tracing trong các ứng dụng LLM
- Cài đặt và cấu hình LangFuse
- Thực hiện tracing cơ bản với ChatAnthropic
- Tích hợp tracing vào LangChain chains
- Thêm metadata tùy chỉnh cho traces
- Xem và phân tích traces trên LangFuse UI

## 1. Giới thiệu về LangFuse Tracing

### Tracing là gì?

**Tracing** trong LangFuse là quá trình ghi lại và theo dõi toàn bộ flow của các lời gọi LLM trong ứng dụng của bạn. Nó giúp:

- **Debugging**: Xác định nhanh lỗi và vấn đề trong pipeline
- **Performance Monitoring**: Theo dõi latency, token usage, và chi phí
- **Quality Assurance**: Đánh giá chất lượng output và cải thiện prompts
- **User Analytics**: Hiểu cách người dùng tương tác với hệ thống

### Tại sao cần LangFuse?

1. **Observability**: Khả năng quan sát toàn diện vào hệ thống LLM phức tạp
2. **Cost Control**: Theo dõi chi phí sử dụng API theo thời gian thực
3. **Iterative Improvement**: Dữ liệu để cải thiện prompts và chains
4. **Compliance**: Lưu trữ và audit các interactions cho mục đích tuân thủ

## 2. Cài đặt và Cấu hình

### 2.1 Cài đặt các thư viện cần thiết

In [ ]:
# Cài đặt các packages cần thiết
!pip install langfuse langchain langchain-anthropic python-dotenv

### 2.2 Cấu hình biến môi trường

#### Hướng dẫn lấy API Keys từ LangFuse:

1. Truy cập [cloud.langfuse.com](https://cloud.langfuse.com) hoặc self-hosted instance
2. Đăng ký/Đăng nhập vào tài khoản
3. Vào **Settings** → **API Keys**
4. Tạo new API key và copy:
   - **Public Key**: Dùng để identify project
   - **Secret Key**: Dùng để authenticate
   - **Host**: URL của LangFuse instance (mặc định: https://cloud.langfuse.com)

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables từ file .env
load_dotenv()

# Cấu hình LangFuse credentials
os.environ["LANGFUSE_PUBLIC_KEY"] = os.getenv("LANGFUSE_PUBLIC_KEY", "your-public-key")
os.environ["LANGFUSE_SECRET_KEY"] = os.getenv("LANGFUSE_SECRET_KEY", "your-secret-key")
os.environ["LANGFUSE_HOST"] = os.getenv("LANGFUSE_HOST", "https://cloud.langfuse.com")

# Cấu hình Anthropic API key
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY", "your-anthropic-key")

# Verify configuration
print("LangFuse configuration:")
print(f"- Host: {os.environ.get('LANGFUSE_HOST')}")
print(f"- Public Key: {os.environ.get('LANGFUSE_PUBLIC_KEY')[:10]}...")
print("- Secret Key: ***configured***")

### 2.3 Khởi tạo LangFuse client

In [ ]:
from langfuse import Langfuse

# Khởi tạo LangFuse client
langfuse = Langfuse(
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY"),
    host=os.environ.get("LANGFUSE_HOST")
)

# Test connection
try:
    langfuse.auth_check()
    print("✅ Kết nối LangFuse thành công!")
except Exception as e:
    print(f"❌ Lỗi kết nối: {e}")

## 3. Ví dụ 1: Tracing lời gọi ChatAnthropic đơn giản

Trong ví dụ đầu tiên, chúng ta sẽ trace một lời gọi đơn giản đến ChatAnthropic.

In [ ]:
from langchain_anthropic import ChatAnthropic
from langfuse.callback import CallbackHandler

# Khởi tạo callback handler cho LangFuse
langfuse_handler = CallbackHandler(
    public_key=os.environ.get("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.environ.get("LANGFUSE_SECRET_KEY"),
    host=os.environ.get("LANGFUSE_HOST")
)

# Khởi tạo ChatAnthropic với callback
chat = ChatAnthropic(
    model="claude-3-haiku-20240307",
    temperature=0.7,
    callbacks=[langfuse_handler]
)

# Thực hiện một lời gọi đơn giản
response = chat.invoke("Xin chào! Hãy giới thiệu về bản thân bạn trong 2-3 câu.")

print("Response:")
print(response.content)
print("\n---")
print(f"Trace URL: {langfuse_handler.get_trace_url()}")

### Giải thích code:

1. **CallbackHandler**: Đây là bridge giữa LangChain và LangFuse, tự động capture các events
2. **callbacks parameter**: Truyền handler vào model để enable tracing
3. **get_trace_url()**: Lấy URL để xem trace trên LangFuse UI

### Xem trace trên UI:
- Click vào URL được in ra
- Bạn sẽ thấy:
  - Input/Output của lời gọi
  - Latency và token usage
  - Model parameters
  - Timestamps

## 4. Ví dụ 2: Tracing một Chain đơn giản

Trong ví dụ này, chúng ta sẽ tạo một chain với prompt template và trace toàn bộ flow.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Tạo một prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý AI chuyên về {topic}. Hãy trả lời ngắn gọn và chính xác."),
    ("human", "{question}")
])

# Tạo chain với prompt | model | parser
chain = prompt | chat | StrOutputParser()

# Tạo một trace mới cho chain execution
langfuse_handler = CallbackHandler(
    trace_name="Simple Q&A Chain",
    user_id="demo-user-123",
    session_id="session-456"
)

# Thực thi chain với tracing
result = chain.invoke(
    {
        "topic": "lập trình Python",
        "question": "List comprehension trong Python hoạt động như thế nào?"
    },
    config={"callbacks": [langfuse_handler]}
)

print("Chain Result:")
print(result)
print("\n---")
print(f"Trace URL: {langfuse_handler.get_trace_url()}")

### Giải thích chain tracing:

1. **trace_name**: Đặt tên cho trace để dễ tìm kiếm
2. **user_id & session_id**: Metadata để group và filter traces
3. **Chain structure**: LangFuse tự động capture từng step trong chain

### Trong LangFuse UI bạn sẽ thấy:
- **Prompt formatting step**: Input variables được format vào template
- **LLM call**: Request đến Anthropic API
- **Parser step**: Output được parse thành string
- **Nested structure**: Mỗi step được hiển thị như một span riêng

## 5. Ví dụ 3: Tracing với Metadata tùy chỉnh

LangFuse cho phép thêm metadata tùy chỉnh để enrich traces với context bổ sung.

In [ ]:
from datetime import datetime
import time

# Tạo một function với custom tracing
def analyze_text_with_metadata(text: str, analysis_type: str):
    """Phân tích văn bản với metadata phong phú"""
    
    # Tạo trace với metadata
    trace = langfuse.trace(
        name="Text Analysis Pipeline",
        user_id="analyst-001",
        session_id=f"analysis-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
        metadata={
            "analysis_type": analysis_type,
            "text_length": len(text),
            "word_count": len(text.split()),
            "timestamp": datetime.now().isoformat()
        },
        tags=["text-analysis", analysis_type, "demo"]
    )
    
    # Step 1: Pre-processing
    span_preprocess = trace.span(
        name="preprocessing",
        metadata={"step": "clean_text"}
    )
    
    # Simulate preprocessing
    cleaned_text = text.strip().lower()
    span_preprocess.end()
    
    # Step 2: LLM Analysis
    span_llm = trace.span(
        name="llm_analysis",
        metadata={
            "model": "claude-3-haiku",
            "prompt_template": "analysis_v1"
        }
    )
    
    # Create analysis prompt
    analysis_prompt = f"""
    Phân tích văn bản sau theo kiểu '{analysis_type}':
    
    Văn bản: {text}
    
    Yêu cầu:
    1. Tóm tắt nội dung chính
    2. Xác định tone/sentiment
    3. Đưa ra insights chính
    """
    
    # Call LLM with custom handler
    handler = CallbackHandler(
        trace_id=trace.id,
        parent_observation_id=span_llm.id
    )
    
    result = chat.invoke(analysis_prompt, config={"callbacks": [handler]})
    
    span_llm.end(
        output=result.content,
        metadata={
            "response_length": len(result.content),
            "processing_time": "fast"
        }
    )
    
    # Step 3: Post-processing and scoring
    span_scoring = trace.span(
        name="quality_scoring",
        metadata={"scorer_version": "1.0"}
    )
    
    # Simulate quality scoring
    quality_score = 0.85  # Giả định
    
    trace.score(
        name="quality",
        value=quality_score,
        comment="Automated quality assessment"
    )
    
    span_scoring.end(
        metadata={"score": quality_score}
    )
    
    # End trace
    trace.update(
        output=result.content,
        metadata={
            "final_quality_score": quality_score,
            "total_processing_time": "completed"
        }
    )
    
    return {
        "analysis": result.content,
        "quality_score": quality_score,
        "trace_url": trace.get_trace_url()
    }

# Sử dụng function với tracing
sample_text = """
LangFuse là một platform observability mạnh mẽ cho các ứng dụng LLM. 
Nó giúp developers theo dõi, debug và cải thiện các hệ thống AI của họ 
thông qua detailed tracing và analytics.
"""

result = analyze_text_with_metadata(
    text=sample_text,
    analysis_type="technical_summary"
)

print("Analysis Result:")
print(f"Quality Score: {result['quality_score']}")
print(f"\nAnalysis:\n{result['analysis']}")
print(f"\nTrace URL: {result['trace_url']}")

### Giải thích metadata và spans:

1. **Custom Metadata**:
   - `metadata`: Dictionary chứa thông tin bổ sung
   - `tags`: Array để categorize và filter traces
   - Giúp search và analyze traces hiệu quả hơn

2. **Nested Spans**:
   - `trace.span()`: Tạo sub-spans cho từng step
   - Mỗi span có thể có metadata riêng
   - Giúp visualize flow chi tiết

3. **Scoring**:
   - `trace.score()`: Thêm quality metrics
   - Có thể track nhiều scores khác nhau
   - Useful cho A/B testing và optimization

4. **Trace Linking**:
   - `trace_id` và `parent_observation_id`: Link các observations
   - Maintain context qua nhiều operations

## 6. Best Practices và Tips

### 6.1 Batch Operations

In [ ]:
# Ví dụ batch processing với tracing
questions = [
    "Python là gì?",
    "JavaScript khác Python như thế nào?",
    "Khi nào nên dùng TypeScript?"
]

# Process batch với session grouping
batch_session_id = f"batch-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

for i, question in enumerate(questions):
    handler = CallbackHandler(
        trace_name=f"Batch Question {i+1}",
        session_id=batch_session_id,
        tags=["batch", "faq"],
        metadata={
            "question_index": i,
            "batch_size": len(questions)
        }
    )
    
    response = chat.invoke(question, config={"callbacks": [handler]})
    print(f"Q{i+1}: {question}")
    print(f"A: {response.content[:100]}...\n")

### 6.2 Error Handling với Tracing

In [ ]:
def safe_llm_call_with_trace(prompt: str):
    """LLM call với error handling và tracing"""
    trace = langfuse.trace(
        name="Safe LLM Call",
        metadata={"prompt_preview": prompt[:50] + "..."}
    )
    
    try:
        # Attempt LLM call
        handler = CallbackHandler(trace_id=trace.id)
        response = chat.invoke(prompt, config={"callbacks": [handler]})
        
        trace.update(
            output=response.content,
            metadata={"status": "success"}
        )
        return response.content
        
    except Exception as e:
        # Log error to trace
        trace.update(
            output=None,
            metadata={
                "status": "error",
                "error_type": type(e).__name__,
                "error_message": str(e)
            },
            tags=["error"]
        )
        print(f"Error traced: {trace.get_trace_url()}")
        raise

# Test với valid prompt
try:
    result = safe_llm_call_with_trace("Giải thích ngắn gọn về recursion")
    print("Success:", result[:100], "...")
except Exception as e:
    print(f"Failed: {e}")

## 7. Xem và Phân tích Traces trên LangFuse UI

### Navigation cơ bản:

1. **Traces List View**:
   - Filter by: tags, user_id, session_id, date range
   - Sort by: timestamp, latency, cost, score
   - Search by trace name hoặc content

2. **Trace Detail View**:
   - **Timeline**: Visualize span relationships
   - **Metadata panel**: Xem tất cả custom metadata
   - **Input/Output**: Full content của mỗi step
   - **Metrics**: Latency, tokens, cost breakdown

3. **Analytics Dashboard**:
   - **Usage trends**: Token usage over time
   - **Performance**: P50/P95 latency
   - **Cost analysis**: Breakdown by model/user
   - **Score distribution**: Quality metrics

### Pro Tips:
- Sử dụng consistent naming cho traces
- Tag traces theo feature/experiment
- Set up alerts cho errors hoặc high latency
- Export data cho deeper analysis

## 8. Kết luận và Bước tiếp theo

### Tóm tắt những gì đã học:

✅ **Cài đặt và cấu hình LangFuse** với API keys và environment variables

✅ **Tracing cơ bản** với CallbackHandler cho single LLM calls

✅ **Chain tracing** để theo dõi complex pipelines

✅ **Custom metadata và spans** cho detailed observability

✅ **Best practices** cho error handling và batch processing

### Bước tiếp theo:

1. **Advanced Tracing**:
   - Multi-agent systems
   - Streaming responses
   - Async operations

2. **Evaluation & Scoring**:
   - Implement custom scorers
   - A/B testing với traces
   - Quality benchmarking

3. **Integration**:
   - Connect với CI/CD
   - Alerts và monitoring
   - Export cho data analysis

4. **Production Best Practices**:
   - Sampling strategies
   - PII handling
   - Cost optimization

### Resources:
- [LangFuse Documentation](https://langfuse.com/docs)
- [LangChain Integration Guide](https://langfuse.com/docs/integrations/langchain)
- [Example Projects](https://github.com/langfuse/langfuse/tree/main/examples)

In [ ]:
# Cleanup và flush traces
langfuse.flush()
print("✅ Tất cả traces đã được gửi lên LangFuse!")
print("🔗 Truy cập LangFuse UI để xem chi tiết traces của bạn.")